In [1]:
import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd
import math
import numpy as np

ROW_LENGTH = 62
COLUMN_LENGTH = 1.25
NUMBER_OF_ROWS = 8

In [2]:
document_topic_probabilities = pd.read_pickle('../pickles/document_topic_probabilities.pickle')
document_topic_probabilities = pd.read_pickle('../pickles/document_topic_probabilities_temp.pickle')
print document_topic_probabilities[0:10]
document_ids_map = pd.read_pickle('../pickles/doc_id_mapping.pickle').to_dict()
inverted_document_ids_map = {v: int(k) for k, v in document_ids_map.iteritems()}

[[  1.05637098e-03   8.22522229e-08   5.94636994e-02   2.30939023e-01
    4.13362834e-01   4.15827598e-04   2.03155486e-02   8.22538391e-08
    2.55905672e-01   1.85408596e-02]
 [  9.74869374e-03   7.32608673e-08   8.61793606e-02   1.66405010e-01
    4.75424632e-01   7.32606736e-08   1.79472062e-02   3.12553020e-03
    2.07885688e-01   3.32837328e-02]
 [  1.03882003e-07   2.19149155e-06   1.55329877e-02   2.26153761e-01
    4.32700507e-01   4.53022541e-03   2.15439441e-02   1.03887987e-07
    2.46438373e-01   5.30978023e-02]
 [  9.10408958e-08   1.25065518e-04   2.68689394e-02   3.13624985e-01
    3.31090755e-01   2.40107179e-02   1.79918658e-02   4.36032087e-04
    2.65108922e-01   2.07426257e-02]
 [  1.64144633e-03   1.31606586e-02   2.98091602e-02   2.58642731e-01
    3.57738204e-01   7.41416152e-02   1.43787716e-02   7.71439317e-04
    2.04292394e-01   4.54235800e-02]
 [  1.10468429e-07   1.45276426e-02   1.91322418e-02   2.48353011e-01
    4.03651789e-01   2.09628911e-02   9.68380

In [3]:
def calculate_the_sizes(data, number_of_rows, row_length, column_length):
    data_size = len(data)
    single_scan_size = (data_size + 1) / number_of_rows
    column_size = int(math.floor(single_scan_size \
    * (column_length / (row_length + column_length))))
    row_size = single_scan_size - column_size 
    return data_size, single_scan_size, column_size, row_size

In [4]:
def generate_coordinates(data_size, column_size, row_size):    
    x_coord = 0
    y_coord = NUMBER_OF_ROWS - 1
    row_scan = True
    scanning_right = True
    coordinates = {}
    for j in range(0, NUMBER_OF_ROWS):
        for i in range(0, row_size):
            doc_id = i + column_size * j + row_size * j
            coordinates[doc_id] = {'x': x_coord, 'y': y_coord}
            # Switching horizontal scanning to vertical.
            if (i % (row_size - 1) == 0 and i != 0):
                y_coord -= 1
                continue
            if scanning_right:
                x_coord += 1
            else:
                x_coord -= 1         
        # Reversing the scanner's direction.
        scanning_right = not scanning_right
    return coordinates

In [5]:
data_size, single_scan_size, column_size, row_size = \
    calculate_the_sizes(inverted_document_ids_map, NUMBER_OF_ROWS, ROW_LENGTH, COLUMN_LENGTH)
coordinates_mapping = generate_coordinates(data_size, column_size, row_size)

In [38]:
topic_id = 4

probability_matrix = np.zeros(shape=(NUMBER_OF_ROWS, row_size))
for i, doc in enumerate(document_topic_probabilities):
    doc_id = inverted_document_ids_map[i]
#     doc_id = i
    if doc_id in coordinates_mapping:
        doc_coord = coordinates_mapping[doc_id]
        probability_matrix[doc_coord['y']][doc_coord['x']] = doc[topic_id]

In [39]:
import plotly.plotly as py
import plotly.graph_objs as go

x_vector = np.zeros(shape=(row_size, 1))
for i in range(0, row_size):
    x_vector[i] = i
y_vector = np.zeros(shape=(NUMBER_OF_ROWS, 1))
for i in range(0, NUMBER_OF_ROWS):
    y_vector[i] = i

layout = go.Layout(
    title='\'Topic %s\' probability distribution.' % topic_id,
    height=350,
    xaxis=dict(title='x'),
    yaxis=dict(title='y')
)
data = [
    go.Heatmap(
        z=probability_matrix,
        x=x_vector,
        y=y_vector,
        colorscale='Electric',
        colorbar=dict(title='Likelihood')
    )
]
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='topic_%s' % topic_id)